In [ ]:
!git clone https://github.com/ruzanovad/diploma.git

In [ ]:
%ls

In [ ]:
%cd diploma/cnn-rnn

In [ ]:
%pip install evaluate
%pip install pytorch-lightning==2.0.8 

In [ ]:
import pandas as pd
import torch
from torch.utils.checkpoint import checkpoint
from torch.utils.data import DataLoader, Dataset
from torch import nn, Tensor
# import torchdata.datapipes as dp
import matplotlib.pyplot as plt
from model import Image2Latex, Text
from pathlib import Path
import torchvision
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
import glob
from tqdm.notebook import tqdm
import time
from nltk.metrics import edit_distance
from typing import Tuple
import pytorch_lightning as pl

In [ ]:
data_path = Path('/kaggle/input/im2latex100k')
img_path = Path('/kaggle/input/im2latex100k/formula_images_processed/formula_images_processed')

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir ../tb_logs/
# %tensorboard --logdir tb_logs/ --bind_all
# %tensorboard --logdir=your_log_dir --bind_all

In [ ]:
%run main.py --batch-size 256 --accumulate-batch 512 --train --log-step 1 \
--num-workers 2 \
--max-epochs 40 --lr 0.01 --model-name conv_bilstm_lstm_train --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed \
--data-path /kaggle/input/im2latex100k \
--vocab_file data/vocab/latex_tokens.json \
--predict-img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png \
--notebook --gpu

In [ ]:
!nvidia-smi

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
%run main.py --batch-size 4 --accumulate-batch 8 --train --log-step 1 \
--train-sample 5 --test-sample 0 --val-sample 0 --num-workers 2 \
--max-epochs 40 --lr 0.01 --model-name conv_bilstm_lstm_predict --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path data/im2latex100k/formula_images_processed/formula_images_processed \
--data-path data/im2latex100k --vocab_file data/vocab/latex_tokens.json \
--predict-img-path data/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png \
--ckpt-path tb_logs/kaggle_20epoch/checkpoints/model-19-1.10.ckpt --num-layers 2

--notebook

In [ ]:
#!/bin/bash
python main.py --batch-size 128 --accumulate-batch 256 --train --log-step 64 --log-text \
--num-workers 16 --train-sample 75000 --max-time 00:06:00:00 --val-sample 8000 \
--max-epochs 20 --lr 0.01 --model-name resnet_bilstm_lstm_train --enc-type resnet_row_encoder \
--enc-dim 512 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.4 \
--decode-type beamsearch --beam-width 5 --grad-clip 1.0 --dataset 100k \
--img-path data/im2latex100k/formula_images_processed/formula_images_processed \
--data-path data/im2latex100k \
--vocab_file data/vocab/latex_tokens.json \
--predict-img-path data/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png \
--gpu --ckpt-path checkpoints/model-15-0.64.ckpt  